In [7]:
from dotenv import load_dotenv
import os

load_dotenv()
dbconn = os.getenv("DBCONN")


In [11]:
import psycopg

def create_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    cur.execute(
        '''
            CREATE TABLE IF NOT EXISTS api_data(
                date TIMESTAMP PRIMARY KEY,
                open FLOAT,
                close FLOAT
            );
        '''
    )
    conn.commit()
    cur.close()
    conn.close()

create_table()


In [16]:
import psycopg
from dotenv import load_dotenv
import os

load_dotenv()
dbconn = os.getenv("DBCONN")

def create_news_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    cur.execute('''
        CREATE TABLE IF NOT EXISTS news_data (
            id SERIAL PRIMARY KEY,
            date TEXT,
            title TEXT,
            url TEXT
        );
    ''')

    conn.commit()
    cur.close()
    conn.close()

create_news_table()


In [17]:
def insert_news_entry(date, title, url):
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    cur.execute(
        '''
        INSERT INTO news_data (date, title, url)
        VALUES (%s, %s, %s);
        ''',
        (date, title, url)
    )

    conn.commit()
    cur.close()
    conn.close()

In [18]:
import requests
from bs4 import BeautifulSoup

url = "https://u.today/bitcoin-news"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

articles = soup.find_all('div', class_='news__item')

results = []

for article in articles:
    title_tag = article.find('div', class_='news__item-title')
    link_tag = article.find('a', class_='news__item-body')
    date_tag = article.find('div', class_='humble')

    if title_tag and link_tag and date_tag:
        title = title_tag.get_text(strip=True)
        link = "https://u.today" + link_tag['href']
        date = date_tag.get_text(strip=True)
        results.append((date, title, link))


In [19]:
import psycopg
from dotenv import load_dotenv
import os

load_dotenv()
dbconn = os.getenv("DBCONN")

def insert_news_data(data):
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    for date, title, url in data:
        cur.execute(
            '''
            INSERT INTO news_data (date, title, url)
            VALUES (%s, %s, %s)
            ON CONFLICT DO NOTHING;  -- falls du duplizierte Daten vermeiden willst
            ''',
            (date, title, url)
        )

    conn.commit()
    cur.close()
    conn.close()

# Ausführen
insert_news_data(results)


In [20]:
import os
import psycopg
from datetime import datetime
from dotenv import load_dotenv

# .env Datei laden
load_dotenv()
dbconn = os.getenv("DBCONN")

# Verbindung aufbauen
conn = psycopg.connect(dbconn)
cur = conn.cursor()

# Tabelle erstellen
cur.execute('''
    CREATE TABLE IF NOT EXISTS news_data (
        id SERIAL PRIMARY KEY,
        date TEXT,
        title TEXT,
        url TEXT
    );
''')

# Beispiel-Daten einfügen
cur.execute('''
    INSERT INTO news_data (date, title, url)
    VALUES (%s, %s, %s);
''', (
    "2025-06-02",
    "Bitcoin price reaches new record",
    "https://u.today/bitcoin-price-record"
))

# Änderungen speichern
conn.commit()

# Einträge anzeigen (Test)
cur.execute("SELECT * FROM news_data ORDER BY id DESC LIMIT 5;")
rows = cur.fetchall()
for row in rows:
    print(row)

# Verbindung schließen
cur.close()
conn.close()


(46, '2025-06-02', 'Bitcoin price reaches new record', 'https://u.today/bitcoin-price-record')
(45, 'Apr 1, 2025 - 12:25', 'Top Crypto Exchanges 2025: Full Guide', 'https://u.todayhttps://u.today/guides/top-crypto-exchanges-2025-full-guide')
(44, 'Apr 16, 2025 - 12:35', 'Gate.io Top Crypto Exchange Makes Digital Assets Accessible: Review', 'https://u.todayhttps://u.today/reviews/gateio-top-crypto-exchange-makes-digital-assets-accessible-review')
(43, 'May 2, 2025 - 18:50', 'Crypto App Triv Ensures Secure Web3 Journey With Triv Shield Instrument: Review', 'https://u.todayhttps://u.today/reviews/crypto-app-triv-ensures-secure-web3-journey-with-triv-shield-instrument-review')
(42, 'May 5, 2025 - 7:56', 'How to Buy Ethereum: Step-by-Step Guide', 'https://u.todayhttps://u.today/guides/how-to-buy-ethereum-steb-by-step-guide')


In [ ]:
# This script fetches cryptocurrency prices from an API and stores them in a PostgreSQL database using Supabase.
# It creates a table if it doesn't exist and inserts the data, handling conflicts gracefully.   
import requests
import psycopg
from dotenv import load_dotenv
import os

# .env Datei laden (lokal)
load_dotenv()
dbconn = os.getenv("DBCONN")

# API-Anfrage
url = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey=demo"
response = requests.get(url)
data = response.json()

# Zeitreihen-Daten extrahieren
timeseries = data["Time Series (Digital Currency Daily)"]

# Verbindung zu Supabase PostgreSQL
conn = psycopg.connect(dbconn)
cur = conn.cursor()

# Tabelle erstellen (falls nicht vorhanden)
cur.execute('''
    CREATE TABLE IF NOT EXISTS crypto_prices (
        date TEXT PRIMARY KEY,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume FLOAT
    );
''')

# Daten einfügen
for date, values in timeseries.items():
    cur.execute('''
        INSERT INTO crypto_prices (date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (date) DO NOTHING;
    ''', (
        date,
        float(values["1. open"]),
        float(values["2. high"]),
        float(values["3. low"]),
        float(values["4. close"]),
        float(values["5. volume"])
    ))

# Speichern und schließen
conn.commit()
cur.close()
conn.close()


In [ ]:
#sqlite3 to_csv
import sqlite3
import pandas as pd

conn = sqlite3.connect("crypto_data.db")
df = pd.read_sql_query("SELECT * FROM crypto_prices", conn)
df.to_csv("crypto_prices.csv", index=False)
conn.close()
